In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-CSE-CIC-IDS2018-v2.parquet"
data = pd.read_parquet(file_name)

In [3]:
##ADDED
file_name_test = "Self-Collected-NF-CSE-CICIDS.parquet"
test_data = pd.read_parquet(file_name_test)
##

In [4]:
data.Label.value_counts()

Label
0    16635567
1     2258141
Name: count, dtype: int64

In [5]:
##ADDED
test_data.Label.value_counts()
##

Label
0    636608
1    349522
Name: count, dtype: int64

In [6]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [7]:
##ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [8]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [9]:
##ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [10]:
data.Attack.unique()

array(['SSH-Bruteforce', 'Benign', 'DDoS attacks-LOIC-HTTP',
       'DDOS attack-HOIC', 'DoS attacks-Slowloris', 'DoS attacks-Hulk',
       'FTP-BruteForce', 'Infilteration', 'Bot', 'DoS attacks-GoldenEye',
       'Brute Force -Web', 'DoS attacks-SlowHTTPTest', 'SQL Injection',
       'DDOS attack-LOIC-UDP', 'Brute Force -XSS'], dtype=object)

In [11]:
##ADDED
test_data.Attack.unique()
##

array(['DoS attacks-GoldenEye', 'Benign', 'FTP-BruteForce',
       'DoS attacks-Hulk', 'Infilteration', 'DoS attacks-SlowHTTPTest',
       'DoS attacks-Slowloris', 'SQL Injection', 'SSH-Bruteforce',
       'Brute Force -Web', 'Brute Force -XSS'], dtype=object)

In [12]:
data = data.groupby(by='Attack').sample(frac=0.02, random_state=13)

In [13]:
##ADDED
test_data = test_data.groupby(by='Attack').sample(frac=0.5, random_state=13)
##

In [14]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711,...,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711
Bot,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862,...,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862
Brute Force -Web,43,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
Brute Force -XSS,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
DDOS attack-HOIC,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617,...,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617
DDOS attack-LOIC-UDP,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
DDoS attacks-LOIC-HTTP,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146,...,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146
DoS attacks-GoldenEye,554,554,554,554,554,554,554,554,554,554,...,554,554,554,554,554,554,554,554,554,554
DoS attacks-Hulk,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653,...,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653


In [15]:
##ADDED
test_data.groupby(by="Attack").count()
##

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,318304,318304,318304,318304,318304,318304,318304,318304,318304,318304,...,318304,318304,318304,318304,318304,318304,318304,318304,318304,318304
Brute Force -Web,14696,14696,14696,14696,14696,14696,14696,14696,14696,14696,...,14696,14696,14696,14696,14696,14696,14696,14696,14696,14696
Brute Force -XSS,8396,8396,8396,8396,8396,8396,8396,8396,8396,8396,...,8396,8396,8396,8396,8396,8396,8396,8396,8396,8396
DoS attacks-GoldenEye,48934,48934,48934,48934,48934,48934,48934,48934,48934,48934,...,48934,48934,48934,48934,48934,48934,48934,48934,48934,48934
DoS attacks-Hulk,28732,28732,28732,28732,28732,28732,28732,28732,28732,28732,...,28732,28732,28732,28732,28732,28732,28732,28732,28732,28732
DoS attacks-SlowHTTPTest,16961,16961,16961,16961,16961,16961,16961,16961,16961,16961,...,16961,16961,16961,16961,16961,16961,16961,16961,16961,16961
DoS attacks-Slowloris,25778,25778,25778,25778,25778,25778,25778,25778,25778,25778,...,25778,25778,25778,25778,25778,25778,25778,25778,25778,25778
FTP-BruteForce,3601,3601,3601,3601,3601,3601,3601,3601,3601,3601,...,3601,3601,3601,3601,3601,3601,3601,3601,3601,3601
Infilteration,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,...,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000


In [16]:
# X = data.drop(columns=["Attack", "Label"])
# y = data[["Attack", "Label"]]

# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=13, stratify=y)
# ADDED
X = test_data.drop(columns=["Attack", "Label"])
y = test_data[["Attack", "Label"]]
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)
##

In [17]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [18]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [19]:
# (Modified)
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()
X_train['id'] = X_train.index

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()
X_test['id'] = X_test.index

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [20]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h,id
223488,145.82.199.44,182.129.67.88,1.561411e-06,2.992190e-08,0.007700,7.489043e-05,0.005921,5.286383e-05,1.292429e-07,8.974047e-08,...,0.002441,0.002392,1.561411e-06,1.561411e-06,1.561411e-06,1.561411e-06,-4.405319e-06,1.621889e-06,"[1.561410674941746e-06, 2.992189736595837e-08,...",223488
536151,103.57.14.23,182.129.67.88,1.213097e-07,1.148535e-07,0.000444,1.711299e-06,0.000492,6.845196e-07,5.057894e-08,5.229720e-08,...,0.000196,0.000171,1.213097e-07,1.213097e-07,1.213097e-07,1.213097e-07,-3.422598e-07,1.260084e-07,"[1.2130973390332713e-07, 1.1485351939588204e-0...",536151
474890,145.82.199.44,182.129.67.88,1.935543e-07,1.832531e-07,0.000750,2.730443e-06,0.000785,1.092177e-06,8.070062e-08,8.344217e-08,...,0.000313,0.000274,1.935543e-07,1.935543e-07,1.935543e-07,1.935543e-07,-5.460885e-07,2.010512e-07,"[1.93554282718865e-07, 1.8325314753491098e-07,...",474890
26566,145.82.199.44,190.201.33.51,1.044272e-08,9.886946e-09,0.000006,8.838835e-08,0.000000,0.000000e+00,1.786158e-08,1.782078e-08,...,0.000015,0.000000,1.044272e-08,1.044272e-08,1.044272e-08,1.044272e-08,-2.946278e-08,1.084719e-08,"[1.0442716994359184e-08, 9.886946086396818e-09...",26566
158524,190.201.33.51,103.57.14.23,9.990043e-10,9.458364e-10,0.000011,8.455686e-09,0.000000,0.000000e+00,4.165253e-10,4.306755e-10,...,0.000001,0.000000,9.990043e-10,9.990043e-10,9.990043e-10,9.990043e-10,-2.818562e-09,1.037699e-09,"[9.990043163782575e-10, 9.458363969304953e-10,...",158524


In [21]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [22]:
# Training graph (Modified)

train['id'] = train.index

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

test['id'] = test.index

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [23]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [24]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [25]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [26]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [27]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [28]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [29]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [30]:
# Convert to GPU
train_g = train_g.to('cuda')

In [31]:
dgi.load_state_dict(torch.load('best_dgi_CSE_v2.pkl'))

/tmp/ipykernel_48915/1688994637.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dgi.load_state_dict(torch.load('best_dgi_CSE_v2.pkl'))


<All keys matched successfully>

In [32]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [33]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [34]:
# Convert to GPU
test_g = test_g.to('cuda')

In [35]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [36]:
# Multimodal (Fusion) Learning

df_train = pd.DataFrame(training_emb,)
# map the id to the original data
df_train['id'] = train_g.edata['id'].detach().cpu().numpy()


df_raw_train = pd.DataFrame(X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_fuse_train = pd.merge(df_train, df_raw_train, on='id', how='left')
df_fuse_train = df_fuse_train.drop(columns=["id"])
df_fuse_train["Attacks"] = train_g.edata['Attack'].detach().cpu().numpy()
df_fuse_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb,)
# map the id to the original data
df_test['id'] = test_g.edata['id'].detach().cpu().numpy()

df_raw_test = pd.DataFrame(X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_raw_test = pd.merge(df_test, df_raw_test, on='id', how='left')
df_fuse_test = df_raw_test.drop(columns=["id"])
df_fuse_test["Attacks"] = test_g.edata['Attack'].detach().cpu().numpy()
df_fuse_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

# Embeddings CBLOF  Embeddings

In [37]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [38]:
benign_fuse_train_samples = df_fuse_train[df_fuse_train.Label == 0].drop(columns=["Label", "Attacks"])
normal_fuse_train_samples = df_fuse_train.drop(columns=["Label", "Attacks"])

fuse_train_labels = df_fuse_train["Label"]
fuse_test_labels = df_fuse_test["Label"]

fuse_test_samples = df_fuse_test.drop(columns=["Label", "Attacks"])

In [39]:
fuse_test_samples

,0,1,2,3,4,5,6,7,8,9,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
0,-0.006735,-0.015655,0.060072,0.016466,0.103703,-0.035008,-0.054476,0.104884,0.075570,0.086115,...,0.000000,0.000000,0.000108,0.000071,5.051694e-08,5.051694e-08,5.051694e-08,5.051694e-08,-1.425271e-07,5.247361e-08
1,-0.006735,-0.015655,0.060072,0.016466,0.103703,-0.035008,-0.054476,0.104884,0.075570,0.086115,...,0.000000,0.000000,0.000000,0.000000,3.600095e-08,3.600095e-08,3.600095e-08,3.600095e-08,-1.015721e-07,3.739537e-08
2,-0.006735,-0.015655,0.060072,0.016466,0.103703,-0.035008,-0.054476,0.104884,0.075570,0.086115,...,0.000000,0.000002,0.001242,0.001086,7.682698e-07,7.682698e-07,7.682698e-07,7.682698e-07,-2.167574e-06,7.980271e-07
3,-0.006735,-0.015655,0.060072,0.016466,0.103703,-0.035008,-0.054476,0.104884,0.075570,0.086115,...,0.000000,0.000005,0.001451,0.001269,8.974479e-07,8.974479e-07,8.974479e-07,8.974479e-07,-2.532034e-06,9.322087e-07
4,-0.006735,-0.015655,0.060072,0.016466,0.103703,-0.035008,-0.054476,0.104884,0.075570,0.086115,...,0.000000,0.000000,0.014156,0.014358,7.810294e-08,7.810294e-08,7.810294e-08,7.810294e-08,-2.203574e-07,8.112809e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295835,-0.008067,-0.003563,0.047126,0.016540,0.082543,-0.018900,-0.057021,0.095504,0.069171,0.065528,...,0.000000,0.000000,0.891492,0.000000,4.821519e-06,4.821519e-06,4.821519e-06,4.821519e-06,-1.360330e-05,5.008270e-06
295836,-0.008067,-0.003563,0.047126,0.016540,0.082543,-0.018900,-0.057021,0.095504,0.069171,0.065528,...,0.000000,0.000000,0.000155,0.000000,8.558630e-10,8.558630e-10,8.558630e-10,8.558630e-10,-2.414707e-09,8.890130e-10
295837,-0.008067,-0.003563,0.047126,0.016540,0.082543,-0.018900,-0.057021,0.095504,0.069171,0.065528,...,0.000000,0.000000,0.224149,0.000000,1.212280e-06,1.212280e-06,1.212280e-06,1.212280e-06,-3.420292e-06,1.259235e-06
295838,-0.008067,-0.003563,0.047126,0.016540,0.082543,-0.018900,-0.057021,0.095504,0.069171,0.065528,...,0.000000,0.000000,0.279151,0.000000,1.509750e-06,1.509750e-06,1.509750e-06,1.509750e-06,-4.259567e-06,1.568228e-06


In [40]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [41]:
from pyod.models.cblof import CBLOF
n_est = [5,6,7,9,10] # cant be lower than 5 or 4
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [02:54<00:00,  5.80s/it]

{'n_estimators': 10, 'con': 0.1}
0.6155932390367447
              precision    recall  f1-score   support

           0     0.7101    0.9002    0.7939    190984
           1     0.6452    0.3307    0.4373    104856

    accuracy                         0.6983    295840
   macro avg     0.6777    0.6154    0.6156    295840
weighted avg     0.6871    0.6983    0.6675    295840



In [42]:
n_est = [5,6,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [04:59<00:00,  9.98s/it]

{'n_estimators': 7, 'con': 0.1}
0.45927050718610346
              precision    recall  f1-score   support

           0     0.6477    0.9030    0.7543    190984
           1     0.3733    0.1053    0.1642    104856

    accuracy                         0.6202    295840
   macro avg     0.5105    0.5041    0.4593    295840
weighted avg     0.5504    0.6202    0.5452    295840



In [43]:
# HBOS  Embeddings+Raw (Multimodal)

In [44]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [45]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [03:09<00:00,  5.26s/it]

{'n_estimators': 20, 'con': 0.01}
0.488837168435446
              precision    recall  f1-score   support

           0     0.6399    0.5453    0.5889    190984
           1     0.3476    0.4412    0.3888    104856

    accuracy                         0.5084    295840
   macro avg     0.4938    0.4932    0.4888    295840
weighted avg     0.5363    0.5084    0.5180    295840



In [46]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [04:00<00:00,  6.68s/it]

{'n_estimators': 5, 'con': 0.04}
0.4800184153169452
              precision    recall  f1-score   support

           0     0.6352    0.7056    0.6686    190984
           1     0.3283    0.2621    0.2915    104856

    accuracy                         0.5484    295840
   macro avg     0.4818    0.4838    0.4800    295840
weighted avg     0.5264    0.5484    0.5349    295840



In [47]:
##  PCA  Emb+Raw (Multimodal/Fusion) Learning

In [48]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [02:28<00:00,  4.14s/it]

{'n_estimators': 15, 'con': 0.04}
0.47515125592512675
              precision    recall  f1-score   support

           0     0.6342    0.4725    0.5416    190984
           1     0.3439    0.5037    0.4088    104856

    accuracy                         0.4836    295840
   macro avg     0.4891    0.4881    0.4752    295840
weighted avg     0.5313    0.4836    0.4945    295840



In [49]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [03:23<00:00,  5.66s/it]

{'n_estimators': 5, 'con': 0.1}
0.45924473758167716
              precision    recall  f1-score   support

           0     0.6206    0.4412    0.5157    190984
           1     0.3333    0.5088    0.4028    104856

    accuracy                         0.4651    295840
   macro avg     0.4770    0.4750    0.4592    295840
weighted avg     0.5188    0.4651    0.4757    295840



In [50]:
##  IF  Emb+Raw

In [51]:
for i in range(len(benign_fuse_train_samples.columns)):
    benign_fuse_train_samples.rename(columns={benign_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(normal_fuse_train_samples.columns)):
    normal_fuse_train_samples.rename(columns={normal_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(fuse_test_samples.columns)):
    fuse_test_samples.rename(columns={fuse_test_samples.columns[i]: f"feature {i}"}, inplace=True)

In [52]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples.to_numpy())
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  4%|▍         | 1/24 [00:01<00:29,  1.30s/it]/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  8%|▊         | 2/24 [00:02<00:28,  1.31s/it]/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 12%|█▎        | 3/24 [00:03<00:26,  1.28s/it]/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 17%|█▋        | 4/24 [00:05<00:25,  1.28s/it]/home/kienho/minif

{'n_estimators': 20, 'con': 0.01}
0.40237241397290724
              precision    recall  f1-score   support

           0     0.6461    0.9909    0.7822    190984
           1     0.4117    0.0116    0.0225    104856

    accuracy                         0.6438    295840
   macro avg     0.5289    0.5012    0.4024    295840
weighted avg     0.5630    0.6438    0.5130    295840



In [53]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 24/24 [01:08<00:00,  2.85s/it]

{'n_estimators': 100, 'con': 0.01}
0.40169676106086727
              precision    recall  f1-score   support

           0     0.6449    0.9843    0.7792    190984
           1     0.3062    0.0126    0.0242    104856

    accuracy                         0.6399    295840
   macro avg     0.4755    0.4985    0.4017    295840
weighted avg     0.5248    0.6399    0.5116    295840

